# Making LAT tables for both

In [5]:
def parity(x):
    return bin(x).count("1") % 2

def compute_bias_LAT(sbox):
    n = (len(sbox) - 1).bit_length()
    size = 1 << n
    half = size // 2

    bias = [[0]*size for _ in range(size)]

    for a in range(size):            # all input masks
        for b in range(size):        # all output masks
            count = 0
            for x in range(size):    # all inputs
                y = sbox[x]
                if parity(a & x) == parity(b & y):
                    count += 1
            bias[a][b] = count - half   # convert to bias (may be negative)

    return bias


def print_LAT(lat):
    size = len(lat)
    print("     ", end="")
    for b in range(size):
        print(f"{b:4}", end="")
    print("\n" + "-"*(5 + 4*size))

    for a in range(size):
        print(f"{a:3} |", end=" ")
        for b in range(size):
            print(f"{lat[a][b]:3}", end=" ")
        print()


# -----------------------------
# Example S-box (replace with yours)
sbox1 = [0, 6,14, 1,15, 4, 7,13, 9, 8,12, 5, 2,10, 3,11]
sbox2 = [0, 9,13, 2,15, 1,11, 7, 6, 4, 5, 3, 8,12,10,14]
bias_lat1 = compute_bias_LAT(sbox1)
bias_lat2 = compute_bias_LAT(sbox2)
print("LAT for S-box 1:")
print_LAT(bias_lat1)
print("\nLAT for S-box 2:")
print_LAT(bias_lat2)


LAT for S-box 1:
        0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15
---------------------------------------------------------------------
  0 |   8   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 
  1 |   0   0  -2   2   0   0   2  -2   0   0  -2   2   4   4  -2   2 
  2 |   0   4   0   0   2   2   2  -2   0   4   0   0  -2  -2  -2   2 
  3 |   0  -4   2  -2   2   2   0   0   0   4   2  -2   2   2   0   0 
  4 |   0   2   4  -2   0  -2   4   2   0  -2   0  -2   0   2   0   2 
  5 |   0   2   2   0   0  -2  -2   0  -4   2   2   4   0   2   2   0 
  6 |   0  -2   0   2   2   0   2   4   0   2  -4   2  -2   0   2   0 
  7 |   0  -2   2   0   2   0   0  -2   4  -2   2   4  -2   0   0   2 
  8 |   0   0   0   0  -4   0   4   0   2   2   2   2   2  -2   2  -2 
  9 |   0   0  -2   2   0   4   2   2  -2  -2   4   0  -2   2   0   0 
 10 |   0   0   0   4  -2  -2  -2   2   2   2   2  -2   0   0   0   4 
 11 |   0   0   2   2   2   2   0   0  -2  -2   0   0   4  -4 

### Analysis on it

In [7]:
import numpy as np

# LAT is your 16x16 bias table (after subtracting 8)
LAT = np.array(bias_lat1)

# 1. Maximum absolute bias
abs_max = np.max(np.abs(LAT[1:]))
positions = np.argwhere(np.abs(LAT) == abs_max)

print("Maximum absolute bias:", abs_max)
print("Occurs at (a, b):", positions.tolist())

# 2. Nonlinearity
nonlinearity = 8 - abs_max // 2
print("Nonlinearity:", nonlinearity)

# 3. Zero-bias count
zero_count = np.sum(LAT == 0)
print("Number of zero-bias entries:", zero_count)

# 4. Row-wise min/max bias
for a in range(16):
    print(f"Row {a}: min={LAT[a].min()}, max={LAT[a].max()}")

# 5. Check for linear structures
for a in range(16):
    if np.all(LAT[a] == 0):
        print(f"Row {a} is all zero → linear structure!")

for b in range(16):
    if np.all(LAT[:,b] == 0):
        print(f"Column {b} is all zero → linear structure!")


Maximum absolute bias: 4
Occurs at (a, b): [[1, 12], [1, 13], [2, 1], [2, 9], [3, 1], [3, 9], [4, 2], [4, 6], [5, 8], [5, 11], [6, 7], [6, 10], [7, 8], [7, 11], [8, 4], [8, 6], [9, 5], [9, 10], [10, 3], [10, 15], [11, 12], [11, 13], [12, 2], [12, 4], [13, 7], [13, 14], [14, 5], [14, 14], [15, 3], [15, 15]]
Nonlinearity: 6
Number of zero-bias entries: 105
Row 0: min=0, max=8
Row 1: min=-2, max=4
Row 2: min=-2, max=4
Row 3: min=-4, max=4
Row 4: min=-2, max=4
Row 5: min=-4, max=4
Row 6: min=-4, max=4
Row 7: min=-2, max=4
Row 8: min=-4, max=4
Row 9: min=-2, max=4
Row 10: min=-2, max=4
Row 11: min=-4, max=4
Row 12: min=-4, max=4
Row 13: min=-4, max=4
Row 14: min=-2, max=4
Row 15: min=-4, max=4


In [8]:
import numpy as np

# LAT is your 16x16 bias table (after subtracting 8)
LAT = np.array(bias_lat2)

# 1. Maximum absolute bias
abs_max = np.max(np.abs(LAT[1:]))
positions = np.argwhere(np.abs(LAT) == abs_max)

print("Maximum absolute bias:", abs_max)
print("Occurs at (a, b):", positions.tolist())

# 2. Nonlinearity
nonlinearity = 8 - abs_max // 2
print("Nonlinearity:", nonlinearity)

# 3. Zero-bias count
zero_count = np.sum(LAT == 0)
print("Number of zero-bias entries:", zero_count)

# 4. Row-wise min/max bias
for a in range(16):
    print(f"Row {a}: min={LAT[a].min()}, max={LAT[a].max()}")

# 5. Check for linear structures
for a in range(16):
    if np.all(LAT[a] == 0):
        print(f"Row {a} is all zero → linear structure!")

for b in range(16):
    if np.all(LAT[:,b] == 0):
        print(f"Column {b} is all zero → linear structure!")


Maximum absolute bias: 4
Occurs at (a, b): [[1, 5], [1, 7], [2, 2], [2, 6], [3, 2], [3, 6], [4, 8], [4, 9], [5, 4], [5, 14], [6, 11], [6, 12], [7, 4], [7, 14], [8, 1], [8, 9], [9, 3], [9, 12], [10, 10], [10, 15], [11, 5], [11, 7], [12, 1], [12, 8], [13, 11], [13, 13], [14, 3], [14, 13], [15, 10], [15, 15]]
Nonlinearity: 6
Number of zero-bias entries: 105
Row 0: min=0, max=8
Row 1: min=-2, max=4
Row 2: min=-2, max=4
Row 3: min=-4, max=4
Row 4: min=-2, max=4
Row 5: min=-4, max=4
Row 6: min=-4, max=4
Row 7: min=-2, max=4
Row 8: min=-4, max=4
Row 9: min=-2, max=4
Row 10: min=-2, max=4
Row 11: min=-4, max=4
Row 12: min=-4, max=4
Row 13: min=-4, max=4
Row 14: min=-2, max=4
Row 15: min=-4, max=4
